In [1]:
#todo KNN imputation if the missing rate is high.
#todo Create new features that may capture underlying patterns (e.g., interaction terms, polynomial features).
#todo create separate file for data preparation

In [2]:
##
import pandas as pd
import numpy as np
import warnings

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from TargetEncoder import TargetEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [3]:
warnings.filterwarnings('ignore')

In [4]:
##
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None

In [5]:
##
submission = pd.read_csv('data/sample_submission.csv')
test = pd.read_csv('data/test.csv')
train = pd.read_csv('data/train.csv')
print(train.shape)

(750000, 12)


In [6]:
drop_cols = ['id']
target_col = ['Listening_Time_minutes']
cat_cols = ['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']
num_cols = [col for col in train.columns if col not in drop_cols + cat_cols + target_col]

In [7]:
train.shape

(750000, 12)

In [8]:
X = train.drop(drop_cols + target_col, axis=1)
y = train['Listening_Time_minutes']

In [9]:
X['isnull_Episode_Length_minutes'] = X['Episode_Length_minutes'].isnull().astype(int)

# Impute Episode_Length_minutes  

In [10]:
from sklearn.base import BaseEstimator, RegressorMixin, clone
import numpy as np
import pandas as pd

class SplitByNullPipeline(BaseEstimator, RegressorMixin):
    def __init__(self, pipeline_with, pipeline_without, split_column):
        self.pipeline_with = pipeline_with
        self.pipeline_without = pipeline_without
        self.split_column = split_column

    def fit(self, X, y):
        # Ensure X is a DataFrame
        if not isinstance(X, pd.DataFrame):
            raise ValueError("X must be a pandas DataFrame")

        self.mask = X[self.split_column].isnull()

        # Split
        X_with = X[~self.mask]
        y_with = y[~self.mask]

        X_without = X[self.mask].drop(columns=self.split_column)
        y_without = y[self.mask]

        self.pipeline_with_ = clone(self.pipeline_with).fit(X_with, y_with)
        self.pipeline_without_ = clone(self.pipeline_without).fit(X_without, y_without)
        return self

    def predict(self, X):
        # Ensure X is a DataFrame
        if not isinstance(X, pd.DataFrame):
            raise ValueError("X must be a pandas DataFrame")

        mask = X[self.split_column].isnull()

        X_with = X[~mask]
        X_without = X[mask].drop(columns=self.split_column)

        y_pred = np.empty(X.shape[0])
        y_pred[~mask] = self.pipeline_with_.predict(X_with)
        y_pred[mask] = self.pipeline_without_.predict(X_without)

        return y_pred


In [11]:
# # 1. Split dataset based on presence of Episode_Length_minutes
# has_length = X['Episode_Length_minutes'].notnull()
# no_length = X['Episode_Length_minutes'].isnull()
# 
# X_with = X[has_length].copy()
# y_with = y[has_length].copy()
# 
# X_without = X[no_length].copy()
# y_without = y[no_length].copy()
# 
# # Optional: drop the column for rows where it's missing (to avoid having nulls)
# X_without = X_without.drop(columns=['Episode_Length_minutes'])
# X_with = X_with.fillna(0)  # or use imputed version
# 
# # Optional: handle categorical variables
# X_with = pd.get_dummies(X_with)
# X_without = pd.get_dummies(X_without)
# 
# # Align columns (make sure both have the same features)
# X_with, X_without = X_with.align(X_without, join='left', axis=1, fill_value=0)
# 
# # 2. Train separate models
# model_with = XGBRegressor(random_state=42)
# model_without = XGBRegressor(random_state=42)
# 
# model_with.fit(X_with, y_with)
# model_without.fit(X_without, y_without)
# 
# # 3. Predict
# y_pred_with = model_with.predict(X_with)
# y_pred_without = model_without.predict(X_without)
# 
# # 4. Combine predictions (if needed for evaluation or downstream)
# y_all = np.concatenate([y_pred_with, y_pred_without])
# true_all = np.concatenate([y_with, y_without])
# 
# rmse = np.sqrt(mean_squared_error(true_all, y_all))
# print(f"Combined RMSE: {rmse:.4f}")


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Outliers
 

In [13]:
outliers = 'Episode_Length_minutes > 140 or Number_of_Ads > 3'
index_drop = X_train.query(outliers).index
X_train.drop(index_drop, inplace=True)
y_train.drop(index_drop, inplace=True)

# Data Cleaning and Preprocessing
 

In [14]:
def target_encoder(df_train, y_train, df_test, cols, enc):
    for col in cols:
        df_train.loc[:, col] = enc.fit_transform(df_train.loc[:, col], y_train).reshape(-1)
        df_test.loc[:, col] = enc.transform(df_test.loc[:, col]).reshape(-1)
    df_train[cols] = df_train[cols].apply(pd.to_numeric, errors='coerce')
    df_test[cols] = df_test[cols].apply(pd.to_numeric, errors='coerce')

    return df_train, df_test

In [15]:
def pipe(numer_type_cols, cat_type_cols):
    # Define transformers
    num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy='median')),  # Median imputation
        ("scaler", StandardScaler())  # Standard scaling
    ])

    cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ])

    # Combine preprocessing pipelines
    preprocessor = ColumnTransformer([
        ("num", num_pipeline, numer_type_cols),
        ("cat", cat_pipeline, cat_type_cols)
    ])
    return preprocessor


In [16]:
enc = TargetEncoder()
X_train, X_test = target_encoder(X_train, y_train, X_test, ['Podcast_Name', 'Episode_Title'], enc)

In [17]:
numer_type_cols = X_train.select_dtypes(include=["number"]).columns
cat_type_cols = X_train.select_dtypes(include=["object"]).columns

numer_type_cols_without = [col for col in numer_type_cols if col != 'Episode_Length_minutes']

In [18]:
preprocessor_with = pipe(numer_type_cols, cat_type_cols)
preprocessor_without = pipe(numer_type_cols_without, cat_type_cols)

# Baseline Model:

In [19]:

pipeline_with = Pipeline([
    ("preprocessor", preprocessor_with),
    ("model", XGBRegressor(random_state=42))
])

pipeline_without = Pipeline([
    ("preprocessor", preprocessor_without),
    ("model", XGBRegressor(random_state=42))
])

# Create dual pipeline that branches on nulls
baseline_pipeline = SplitByNullPipeline(
    pipeline_with=pipeline_with,
    pipeline_without=pipeline_without,
    split_column='Episode_Length_minutes'
)

In [20]:
baseline_pipeline.fit(X_train, y_train)

SplitByNullPipeline(pipeline_with=Pipeline(steps=[('preprocessor',
                                                   ColumnTransformer(transformers=[('num',
                                                                                    Pipeline(steps=[('imputer',
                                                                                                     SimpleImputer(strategy='median')),
                                                                                                    ('scaler',
                                                                                                     StandardScaler())]),
                                                                                    Index(['Podcast_Name', 'Episode_Title', 'Episode_Length_minutes',
       'Host_Popularity_percentage', 'Guest_Popularity_percentage',
       'Number_of_Ads', 'isnull_Episode_...
                                                                   importance_type=None,
                                                                   interaction_constraints=None,
                                                                   learning_rate=None,
                                                                   max_bin=None,
                                                                   max_cat_threshold=None,
                                                                   max_cat_to_onehot=None,
                                                                   max_delta_step=None,
                                                                   max_depth=None,
                                                                   max_leaves=None,
                                                                   min_child_weight=None,
                                                                   missing=nan,
                                                                   monotone_constraints=None,
                                                                   multi_strategy=None,
                                                                   n_estimators=None,
                                                                   n_jobs=None,
                                                                   num_parallel_tree=None, ...))]),
                    split_column='Episode_Length_minutes')

In [21]:
# Perform Cross-Validation (5-fold)
cv_rmse = np.sqrt(-cross_val_score(baseline_pipeline, X_train, y_train,
                                   scoring="neg_mean_squared_error", cv=5, n_jobs=-1, verbose=100))

# Train on full training set
baseline_pipeline.fit(X_train, y_train)

# Make predictions on test set
y_pred = baseline_pipeline.predict(X_test)

# Calculate RMSE on test set
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print Results
print(f"Cross-Validation RMSE: {cv_rmse.mean():.4f} ± {cv_rmse.std():.4f}")
print(f"Test Set RMSE: {test_rmse:.4f}")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    6.4s remaining:    9.6s
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    6.5s remaining:    4.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    6.7s finished
Cross-Validation RMSE: 13.0411 ± 0.0234
Test Set RMSE: 12.9834


# Final model for submission

In [22]:
outliers = 'Episode_Length_minutes > 140 or Number_of_Ads > 3'
index_drop = X.query(outliers).index
X.drop(index_drop, inplace=True)
y.drop(index_drop, inplace=True)

In [23]:
enc = TargetEncoder()
X, X_test = target_encoder(X, y, test, ['Podcast_Name', 'Episode_Title'], enc)

In [24]:
baseline_pipeline.fit(X, y)

SplitByNullPipeline(pipeline_with=Pipeline(steps=[('preprocessor',
                                                   ColumnTransformer(transformers=[('num',
                                                                                    Pipeline(steps=[('imputer',
                                                                                                     SimpleImputer(strategy='median')),
                                                                                                    ('scaler',
                                                                                                     StandardScaler())]),
                                                                                    Index(['Podcast_Name', 'Episode_Title', 'Episode_Length_minutes',
       'Host_Popularity_percentage', 'Guest_Popularity_percentage',
       'Number_of_Ads', 'isnull_Episode_...
                                                                   importance_type=None,
                                                                   interaction_constraints=None,
                                                                   learning_rate=None,
                                                                   max_bin=None,
                                                                   max_cat_threshold=None,
                                                                   max_cat_to_onehot=None,
                                                                   max_delta_step=None,
                                                                   max_depth=None,
                                                                   max_leaves=None,
                                                                   min_child_weight=None,
                                                                   missing=nan,
                                                                   monotone_constraints=None,
                                                                   multi_strategy=None,
                                                                   n_estimators=None,
                                                                   n_jobs=None,
                                                                   num_parallel_tree=None, ...))]),
                    split_column='Episode_Length_minutes')

In [25]:
# Make predictions on test set
submission['Listening_Time_minutes'] = baseline_pipeline.predict(X_test)


ValueError: columns are missing: {'isnull_Episode_Length_minutes'}

In [ ]:
submission.to_csv('submission.csv', index=False)